In [22]:
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras.applications import vgg19
from keras import backend as K
from scipy.optimize import fmin_l_bfgs_b
import imageio
import time
from matplotlib import pyplot as plt
import keras
import pydot

In [23]:
# 想要变换的图像的路径
target_image_path = '../data/portrait.png'
# This is the path to the style image.（风格图像的路径）
style_reference_image_path = '../data/popova.png'
width, height = load_img(target_image_path).size
print(width, height)
img_height = 400
img_width = int(width * img_height / height)

251 252


In [24]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))  # 加载图像，归一化大小
    img = img_to_array(img)  # 序列化
    img = np.expand_dims(img, axis=0)  # 展开
    img = vgg19.preprocess_input(img)  # 预处理到0～1
    return img

In [25]:
def deprocess_image(x):
    # vgg19.preprocess_input 的作用是减去 ImageNet 的平均像素值， 使其中心为 0。
    # 这里相当于 vgg19.preprocess_input 的逆操作
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'（将图像由 BGR 格式转换为 RGB 格式。这也是vgg19.preprocess_input 逆操作的一部分）
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [26]:
target_image = K.constant(preprocess_image(target_image_path))
print(target_image.shape)
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
print(style_reference_image.shape)
combination_image = K.placeholder((1, img_height, img_width, 3))
print(combination_image.shape)
# 将三张图像合并为一个批量
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)
print(input_tensor.shape)

(1, 400, 398, 3)
(1, 400, 398, 3)
(1, 400, 398, 3)
(3, 400, 398, 3)


In [27]:
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('Model loaded.')

Model loaded.


In [28]:
# 定义内容损失
# 它要保证目标图像和生成图像在 VGG19 卷积神经网络的顶层具有相似的结果
# 平方和
def content_loss(base, combination):
    return K.sum(K.square(combination - base))
    # sum:计算张量在某一指定轴的和
    # square:元素级的平方操作

In [29]:

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    # 它将批次中的每一个样本展平。
    print('batch_flatten:', features.shape)
    gram = K.dot(features, K.transpose(features))
    # transpose:将张量转置并返回
    # dot:将 2 个张量（和/或变量）相乘并返回一个张量
    print('dot', gram.shape)
    return gram

In [30]:
# 风格损失
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [31]:
# 总变差损失(正则化损失)
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [32]:
model.summary()
keras.utils.vis_utils.pydot = pydot
keras.utils.plot_model(model,show_shapes=True,to_file='../images/vgg19.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [33]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# 用于内容损失的层
content_layer = 'block5_conv2'
# 用于风格损失的层
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# 在定义损失时将所有分量添加到这个标量变量中
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers: #添加每个目标层的风格损失分量
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

batch_flatten: (64, 159200)
dot (64, 64)
batch_flatten: (64, 159200)
dot (64, 64)
batch_flatten: (128, 39800)
dot (128, 128)
batch_flatten: (128, 39800)
dot (128, 128)
batch_flatten: (256, 9900)
dot (256, 256)
batch_flatten: (256, 9900)
dot (256, 256)
batch_flatten: (512, 2450)
dot (512, 512)
batch_flatten: (512, 2450)
dot (512, 512)
batch_flatten: (512, 600)
dot (512, 512)
batch_flatten: (512, 600)
dot (512, 512)


In [34]:
grads = K.gradients(loss, combination_image)[0]
# 返回 combination_image 在 loss 上的梯度

fetch_loss_and_grads = K.function([combination_image], [loss, grads])
# 实例化 Keras 函数

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [36]:
result_prefix = '../images/style_transfer_result/'
iterations = 20

x=preprocess_image(target_image_path)
print(x.shape)
x=x.flatten()
print(x.shape)
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + 'at_iteration_%d.png' % i
    imageio.imwrite(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

(1, 400, 398, 3)
(477600,)
Start of iteration 0
Current loss value: 9388130000.0
Image saved as ../images/style_transfer_result/at_iteration_0.png
Iteration 0 completed in 99s
Start of iteration 1
Current loss value: 3512242200.0
Image saved as ../images/style_transfer_result/at_iteration_1.png
Iteration 1 completed in 106s
Start of iteration 2
Current loss value: 2266319600.0
Image saved as ../images/style_transfer_result/at_iteration_2.png
Iteration 2 completed in 109s
Start of iteration 3
Current loss value: 1760910500.0
Image saved as ../images/style_transfer_result/at_iteration_3.png
Iteration 3 completed in 111s
Start of iteration 4
Current loss value: 1426021500.0
Image saved as ../images/style_transfer_result/at_iteration_4.png
Iteration 4 completed in 105s
Start of iteration 5
Current loss value: 1185838800.0
Image saved as ../images/style_transfer_result/at_iteration_5.png
Iteration 5 completed in 108s
Start of iteration 6
Current loss value: 1045906300.0
Image saved as ../im